#  Scaden-methylation Example

# Getting Started

Before we can start using the pipeline, we need two datasets:
1. A reference dataset to use for training the model. This can be generated in two ways:
    - From a real single nucleus methylation dataset (e.g., generated from single-cell bisulfite sequencing). 
    - From a methylation profile generated from purified samples. This will be used to simulate a single cell dataset.
2. A bulk methylation dataset containing the methylation data of samples that we want to perform deconvolution on.

There may be other required files, depending on which steps of the pipeline we want to run.
Although we focus on methylation data, this program can still work with RNA-seq data like the [original scaden](https://scaden.readthedocs.io/en/latest/usage.html)

## Reference dataset

### From a real single nucleus methylation dataset

If we want to use a real single cell mathylation dataset, it needs to be converted into text files that follow the format of the original [scaden](https://scaden.readthedocs.io/en/latest/usage.html#bulk-simulation). 

Our single cell dataset must have two components: the methylation data for the cells, and the labels for each cell.
The labels will go into one file with a name that ends in "`_celltypes.txt`". This file will have one column called "Celltype" with the celltype labels for each cell, e.g.:
```
Celltype
type1
type2
type9
type4
```

The methylation data will go into another file with a name ending in "`_counts.txt`". This file will have a table of methylation values with cell samples as rows and CpGs as columns, e.g.:
```
     cpg1    cpg2    cpg100  cpg230
0    0.0     1.0     0.5     0.0
1    1.0     0.25    0.0     0.0
2    0.0     0.0     1.0     0.0
3    0.75    0.125   0.2     1.0
```
This table should have row indices that are just numbers starting from 0. The order of the rows should match the order of the cell type labels in the corresponding `_celltypes.txt` file. The two files should also have the same prefix (e.g., `neurons_celltypes.txt` and `neurons_counts.txt` make up one dataset).

See the directory `helper_scripts` for some Python scripts you may find useful for generating this dataset from scBS-seq data.

### From a methylation profile

We can also use a previously created methylation profile generated from purified samples. This consists of a matrix of differentially methylated sites between different cell types. We can sample from this matrix to simulate a single cell dataset.

In [1]:
# Example reference library for peripheral blood
!head -n 15 example/blood_matrix.tsv

	Bas	Bmem	Bn	CD4mem	CD4nv	Tem	Tn	EOS	Mono	Neu	NK	Treg
cg23788058	0.796	0.919	0.94	0.869	0.285	0.897	0.693	0.437	0.949	0.919	0.94	0.9
cg21091766	0.672	0.688	0.858	0.874	0.936	0.458	0.322	0.583	0.811	0.697	0.739	0.872
cg16596052	0.886	0.876	0.888	0.833	0.591	0.826	0.116	0.901	0.936	0.914	0.922	0.858
cg21782213	0.931	0.942	0.945	0.828	0.314	0.935	0.934	0.943	0.953	0.953	0.949	0.919
cg15213052	0.952	0.947	0.949	0.939	0.891	0.935	0.346	0.96	0.968	0.965	0.841	0.949
cg17458390	0.073	0.101	0.077	0.068	0.184	0.114	0.692	0.06	0.055	0.051	0.072	0.077
cg19850895	0.946	0.914	0.94	0.925	0.934	0.869	0.426	0.929	0.96	0.924	0.908	0.946
cg27225309	0.079	0.922	0.585	0.748	0.266	0.518	0.347	0.071	0.351	0.29	0.186	0.179
cg17518643	0.941	0.947	0.954	0.581	0.245	0.887	0.791	0.583	0.929	0.909	0.91	0.719
cg12043508	0.79	0.771	0.881	0.909	0.916	0.881	0.453	0.833	0.823	0.802	0.724	0.923
cg20813776	0.94	0.925	0.931	0.89	0.937	0.386	0.91	0.929	0.937	0.923	0.848	0.924
cg00267793	0.149	0.927	0.921	0.916	0.92	0.921	0

In [2]:
# Sample from the reference matrix to create a simulated single cell dataset
!python helper_scripts/array_to_sc.py -s 100 -r 2 -o 'example/data' example/blood_matrix.tsv

Created files 'example/data_counts.txt' and 'example/data_celltypes.txt'


This script samples each value in the table from a binomial distribution with 1 trial and uses the value as the probability.
The number of successes (0 or 1) is used as the corresponding value in the scMethyl-seq dataset.
The arguments used are:
- `-s`: Numpy random generator seed for reproducibility.
- `-r`: The number of simulated cells to generate for each cell type in the matrix. 
    - By setting this to 2, we created a dataset with 24 cells (2 * 12 cell types). 
    - Higher values (e.g.: 1000) generally give better results.
- `-o`: Set the name and location of the output files.

In [3]:
# Celltype labels file. Each cell type was sampled twice.
!cat example/data_celltypes.txt

Celltype
Bas
Bas
Bmem
Bmem
Bn
Bn
CD4mem
CD4mem
CD4nv
CD4nv
Tem
Tem
Tn
Tn
EOS
EOS
Mono
Mono
Neu
Neu
NK
NK
Treg
Treg


In [4]:
# First 8 columns of our simulated methylation data
!cut -f 1-8 example/data_counts.txt

	cg23788058	cg21091766	cg16596052	cg21782213	cg15213052	cg17458390	cg19850895
0	0	0	1	1	1	0	1
1	1	0	1	1	1	0	1
2	1	0	1	1	1	0	1
3	1	1	1	1	0	1	1
4	0	1	1	1	1	0	1
5	1	1	1	1	1	0	1
6	1	1	1	1	1	0	1
7	0	1	1	1	1	0	1
8	0	1	0	1	1	0	1
9	0	1	1	0	1	0	1
10	0	0	1	1	1	0	0
11	1	1	1	1	1	0	1
12	1	0	0	1	1	1	0
13	1	0	0	1	1	1	1
14	1	1	1	1	1	0	1
15	0	1	1	1	1	0	1
16	0	0	1	1	1	1	1
17	1	1	1	1	1	0	1
18	1	1	1	0	1	0	1
19	1	0	1	1	1	0	1
20	0	1	1	1	1	0	1
21	0	1	1	1	0	0	1
22	1	1	1	0	0	0	1
23	1	0	1	1	1	0	1


## Using the pipeline

There are 5 steps in the pipeline:
1. Simulate training data from single cell dataset
2. Preprocess training data
3. Train model
4. Predict on testing data
5. Evaluate predictions

Each step has its own set of parameters that will be explained below with an example run. 
To run the pipeline:
- Save the parameters to a YAML file, then run `python main.py --load params.yaml` (Recommended)
- Run `python main.py ...` with all the parameters as a single command.

### Pipeline controls and logging

```
  -load TEXT                 YAML file from which parameters are loaded
  -v, --verify               Verify that parameters are valid, then exit
                             without doing any steps.
  -all                       Run all steps of the pipeline (simulate, process,
                             train, predict, evaluate)
  -simulate                  Run scaden simulate
  -process                   Run scaden process
  -train                     Run scaden train
  -predict                   Run scaden predict
  -evaluate                  Run evaluation
  --config TEXT              Name of configuration  [default: test]
  --reference TEXT           Name of the dataset  [default: None]
  --log_params               Create a json file recording the data and model
                             hyperparameters
  --seed INTEGER             Set random seed  [default: 0]
```

For this example, we will go through all steps of the pipeline, so we can just use the `-all` flag. If we only want to run specific steps add each flag (e.g., `-predict -evaluate`) 

We can set some parameters for logging purposes. These parameters will appear in the output logs:
- `--config example`
- `--reference blood`
  - The name of our configuration and our dataset, respectively.
- `--seed 100`
  - The seed that will be used to initialize the NumPy random number generator.

### Simulate

This step will sample from the dataset files and create a h5ad file containing the simualted bulk samples and the ground truth fractions. 

If there is more than one dataset in the `data` directory (i.e., more than one set of x_counts.txt and x_celltypes.txt files), scaden will create a h5ad file for each dataset and a file that combines the datasets.

```
  -o, --out TEXT             Directory to store output files in  [default: ./]
  -d, --data TEXT            Path to scRNA-seq dataset(s)  [default: .]
  -c, --cells INTEGER        Number of cells per sample  [default: 100]
  -n, --n_samples INTEGER    Number of samples to simulate  [default: 1000]
  --pattern TEXT             File pattern to recognize your processed scRNA-
                             seq count files  [default: *_counts.txt]
  -u, --unknown TEXT         Specify cell types to merge into the unknown
                             category. Specify this flag for every cell type
                             you want to merge in unknown.  [default: unknown]
  -p, --prefix TEXT          Prefix to append to training .h5ad file
                             [default: data]
  -f, --data_format TEXT     Data format of scRNA-seq data, can be 'txt' or
                             'h5ad'  [default: txt]

```

Here's what we'll use:
- `--data example/`
  - We previously created our single cell dataset and stored it in the 'example' directory, so that will be our data directory.
- `--out example/`
  - We'll store our simulated training data in the same directory.
- `--cells 100 -n 1000`
  - The training set will contain 1000 samples, each composed of 100 cells.
- `--prefix example_data`
  - The training set will be named 'example_data'.

The result is a file called '`example_data.h5ad`' in our `example/` directory.

## Preprocess

This step will take the training data from the previous step and do three things:
1. Filter the dataset for CpGs that are found in both the training data and the data we are performing deconvolution on.
2. Remove CpGs that have a variance below a specified cutoff.
3. Apply a scaling to the methylation data.

The testing data should be a text file or h5ad file with CpGs in the rows and samples in the columns, e.g.:

In [5]:
!head -n 10 example/testfile.tsv | cut -f 1-8

GSM5527895	GSM5527896	GSM5527897	GSM5527898	GSM5527899	GSM5527900	GSM5527901	GSM5527902
cg23788058	0.798699449	0.784677456	0.74981309	0.789639854	0.810941974	0.825244463	0.788397419
cg21091766	0.687263871	0.730978837	0.743193841	0.645360376	0.69560332	0.72999294	0.737264397
cg16596052	0.879610284	0.833583581	0.841797798	0.735548403	0.830872763	0.844413481	0.76839483
cg21782213	0.868848813	0.878711025	0.833995666	0.862394569	0.877224017	0.882607419	0.846306483
cg15213052	0.889969575	0.887590506	0.885214628	0.864699342	0.865485621	0.882226631	0.884852479
cg17458390	0.117161441	0.080060866	0.071458516	0.143326585	0.08785156	0.074913869	0.090899878
cg19850895	0.878980332	0.875194166	0.902452525	0.805203742	0.875195158	0.873695376	0.844872471
cg27225309	0.438220517	0.302231866	0.421263644	0.335391233	0.251055594	0.336748842	0.322993831
cg17518643	0.817972414	0.76323919	0.700720148	0.810156068	0.819491989	0.796900676	0.79493788


Now we'll set the parameters, which are the following:

```
  --pred TEXT                Bulk data file (or testing set) that we want to
                             perform deconvolution on. Should be a text or
                             h5ad file with rows as genes and samples as
                             columns.
  --training_data TEXT       Training dataset to be processed. Only use this
                             if you are running 'scaden process' by itself.
  --processed_path TEXT      Name of the file that the processed data will be
                             saved to. Must end with .h5ad
  --var_cutoff FLOAT         Filter out genes with a variance less than the
                             specified cutoff. A low value is recommended,
                             this should only remove genes that are obviously
                             uninformative.  [default: 0.1]
  --scaling TEXT             Change scaling option for preprocessing the
                             training data. If something other than the
                             provided options is used, then no scaling will be
                             done. Options: None (No scaling), log /
                             log_min_max (log2, then scale to the range 0,1),
                             frac / fraction (Divide values by the number of
                             cells), frac_notna (Divide values by the number
                             of non-NA values. This must be used with the
                             simulation step and cannot be done by
                             preprocessing separately)  [default: fraction]

```

We will use these parameters:
- `--pred example/testfile.tsv`
    - This file contains the data for the bulk samples that we want to perform deconvolution on.
- `--scaling frac`
    - Fraction scaling, since we are dealing with methylation data of simulated cells
- `--var_cutoff 0`
    - We will assume that the CpGs in our reference methylation array are all important, so we don't want to remove any of them.


The scaling option has 4 options:
- None
- log / log_min_max: Apply log2, then scale to the range 0,1. 
    - This is the default option for RNA-seq data.
- frac / fraction: Divide values by the total number of cells. 
    - This should be used with methyl-seq if our single-cell dataset was simulated from a methylation profile.
- frac_notna: Divide values by the number of non-NA values.
    - This should be used with methyl-seq if we're using a real single-cell dataset with many missing values.

Some parameters are unnecessary or automatically handled if we are going through the entire pipeline. 
- `--training_data`: The h5ad file generated by the simulation step. If we are running the preprocessing separately from the simulation, we need to set this parameter.
- `--processed_path`: Set this if we want to change the name or location of the preprocessed training data By default, the output will be named '`processed.h5ad`' and be in the directory specified with `--out`.


The result is a file (named '`processed.h5ad`' by default) that contains our preprocessed data.

### Training

This step is where the model training is done.

```
  --train_datasets TEXT      Comma-separated list of datasets used for
                             training. Uses all by default.
  --model_dir TEXT           Path to store the model in  [default: ./]
  --batch_size INTEGER       Batch size to use for training.  [default: 128]
  --learning_rate FLOAT      Learning rate used for training.  [default:
                             0.0001]
  --steps INTEGER            Number of training steps.  [default: 5000]
  --loss_values TEXT         Name of file to save text file of loss values
  --loss_curve TEXT          Name of file to save line plot figure of loss
                             values
```

We will use the parameters:
- `--model_dir example/model/`
  - Our model will be stored in the specified directory. This directory must exist before running the training.
- `--batch_size 256`
- `--learning_rate 0.0001`
- `--steps 1000`
  - Various model training hyperparameters.
- `--loss_values example/loss_values.tsv`
  - (OPTIONAL) This will create a text file containing the loss values after each step of training.
- `--loss_curve example/loss_curve.png`
  - (OPTIONAL) This will create a simple plot for the loss values vs step.

If we are running this step separately, we need to specify the following:
- `--processed_path`: Preprocessed training data

The result the creation of a set of files in our model directory.

## Predict

This step will use a trained model to create predictions on bulk samples to predict the cell type proportions.

```
  --prediction_outname TEXT  Name of predictions file  [default:
                             scaden_predictions.txt]
  --prediction_scaling TEXT  Change scaling option for the preprocessing done
                             when making predictions. Uses the same options as
                             --scaling.  [default: fraction]
```

Our parameters:
- `--prediction_outname example/test_predictions.txt`
  - Name of the output text file.
- `--prediction_scaling None`
  - Scaling to apply to the testing data. If this data is a real methylation dataset, don't use any scaling.

If we are running this step separately, we need to specify the following:
- `--model_dir`: Directory to the trained model.
- `--pred`: Bulk samples to perform deconvolution on.
- `--cells`: If we want to use fraction scaling.

The result is a text file containing the predictions.

## Evaluate (WIP)

This step will compare the predictions with the ground truth by calculating the correlation coefficients and mean squared errors.

```
  --ground_truth TEXT        Name of file containing the ground truth cell
                             proportions
```

- `--ground_truth example/test_proportions.tsv`
  - The ground truth proportions should be a table where the columns are cell types and the rows are samples:

The result is a JSON file containing the results of the comparison.

In [6]:
!head example/test_proportions.tsv

bas	bmem	bnv	cd4mem	cd4nv	cd8mem	cd8nv	eos	mono	neu	nk	treg
GSM5527895	13.14	13.26	6.53	10.08	4.47	10.95	5.99	9.01	6.89	3.02	6.36	10.28
GSM5527896	11.54	8.6	6.71	8.65	8.14	9.13	6.16	8.62	3.72	8.63	8.43	11.67
GSM5527897	8.75	11.12	3.68	21.32	10.16	11.84	2.8	5.05	3.92	3.57	5.14	12.68
GSM5527898	8.53	7.2	18.76	4.75	7.9	9.76	14.75	7.05	5.78	3.92	7.35	4.24
GSM5527899	6.98	5.22	5.16	6.04	7.61	8.29	8.19	5.12	8.09	8.28	22.23	8.79
GSM5527900	9.42	12.49	7.14	8.11	5.07	4.9	6.82	10.43	4.96	15.74	9.71	5.21
GSM5527901	2.29	7.85	6.46	8.54	9.55	9.55	9.99	11.26	13.28	9.18	4.52	7.51
GSM5527902	5.56	5.37	7.58	6.23	4.18	5.06	3.83	7.97	10.63	15.69	16.92	10.97
GSM5527903	7.39	7.84	11.17	14.13	8.38	6.5	1.9	10.29	6.38	13.74	8.2	4.08


## Putting everything together

YAML file `params.yaml':
```
all: True
config: example
reference: blood
seed: 100
data: example/
out: example/
cells: 100
n_samples: 1000
prefix: example_data
pred: example/testfile.tsv
scaling: frac
var_cutoff: 0
model_dir: example/model/
batch_size: 256
learning_rate: 0.0001
steps: 1000
loss_values: example/loss_values.tsv
loss_curve: example/loss_curve.png
prediction_outname: example/test_predictions.txt
prediction_scaling: None
ground_truth: example/test_proportions.tsv
```

Then we run the program with these parameters using: `python main.py -load params.yaml`

## Example Run

In [7]:
!python main.py -load example/params.yaml


     ____                _            
    / ___|  ___ __ _  __| | ___ _ __  
    \___ \ / __/ _` |/ _` |/ _ \ '_ \ 
     ___) | (_| (_| | (_| |  __/ | | |
    |____/ \___\__,_|\__,_|\___|_| |_|
    
INFO     Using params from: example/params.yaml                      main.py:112
INFO     Datasets: ['data']                                 bulk_simulator.py:99
INFO     Simulating data from data                         bulk_simulator.py:105
INFO     Loading data dataset ...                          bulk_simulator.py:144
INFO     Merging unknown cell types: ['unknown']           bulk_simulator.py:226
INFO     Subsampling data ...                              bulk_simulator.py:229
Normal samples 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Normal samples 21 ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Normal samples 44 ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Normal samples 68 ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Outputs

In [9]:
!cat example/test_predictions.txt

	Bas	Bmem	Bn	CD4mem	CD4nv	Tem	Tn	EOS	Mono	Neu	NK	Treg
GSM5527895	0.119815744	0.119674735	0.07861956	0.07423564	0.06338039	0.082098626	0.07961836	0.075796954	0.06651795	0.06706738	0.07943547	0.093739145
GSM5527896	0.11470125	0.09604945	0.080230914	0.082268655	0.06662912	0.076183975	0.08212555	0.08213074	0.06365629	0.07871834	0.080222696	0.09708298
GSM5527897	0.10378856	0.1040929	0.07022439	0.10760345	0.06498163	0.09262127	0.08279188	0.063097745	0.057208937	0.06470912	0.06934353	0.11953661
GSM5527898	0.10580767	0.09764365	0.11450633	0.06978834	0.07349116	0.067969695	0.10282135	0.06938842	0.07297294	0.06909428	0.081032775	0.07548339
GSM5527899	0.10259307	0.08286381	0.0727314	0.0781588	0.06294493	0.0780484	0.093173645	0.06892962	0.075281166	0.08531227	0.111167096	0.08879581
GSM5527900	0.10923013	0.10604584	0.08387625	0.06774113	0.063241765	0.06708991	0.07720655	0.08491834	0.07870664	0.102051705	0.08264004	0.07725167
GSM5527901	0.09278202	0.09506547	0.079252005	0.07149267	0.07475855	0.06864

In [10]:
!cat example/report_example.json

{
    "training_time": 68.637,
    "results": {
        "Predictions best correlated celltype": {
            "bas": "Bas",
            "bmem": "Bmem",
            "bnv": "Bn",
            "cd4mem": "CD4mem",
            "cd4nv": "CD4nv",
            "cd8mem": "Tem",
            "cd8nv": "Tn",
            "eos": "EOS",
            "mono": "Mono",
            "neu": "Neu",
            "nk": "NK",
            "treg": "Treg"
        },
        "Correlation coefficient": {
            "bas": 0.9784099842250115,
            "bmem": 0.9186740709042494,
            "bnv": 0.9694224444786744,
            "cd4mem": 0.8976052143901749,
            "cd4nv": 0.6680259811738782,
            "cd8mem": 0.6295658025141089,
            "cd8nv": 0.8458940221241743,
            "eos": 0.8844145633825922,
            "mono": 0.8822245844210457,
            "neu": 0.92459867657708,
            "nk": 0.9716180918400404,
            "treg": 0.749339548692836
        },
        "Mean squared error": {
       

In [11]:
!head example/loss_values.tsv

step	model	loss
0	m256	0.0117640420794487
1	m256	0.009992626495659351
2	m256	0.01273795310407877
3	m256	0.009805307723581791
4	m256	0.011787560768425465
5	m256	0.010207257233560085
6	m256	0.010412284173071384
7	m256	0.01030189823359251
8	m256	0.010286832228302956


**Loss curve:** 

![Loss curve](example/loss_curve.png)

## Required parameters for each step

These arguments must be provided if using individual commands.

Simulate
- `out`: Directory to store output files in.
- `data`: Path to single-cell dataset(s)

Process
- `pred`: Bulk data file that we want to perform deconvolution on.
- `training_data`: Training dataset to be processed, generated by the simulate step.
- `processed_path`: Name of the file that the processed data will be saved to. Must end with .h5ad

Train
- `processed_path`: Name of file containing the data that will be used for training.
- `model_dir`: Path to store trained model.

Predict
- `pred`: Bulk data file that we want to perform deconvolution on.
- `model_dir`: Path to trained model.
